In [28]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import duckdb as dd
import polars as pl
import os
import glob
import shutil
import zipfile
import pickle
from joblib import dump, load
from pathlib import Path
import librosa

In [29]:
# Function to extract features from audio file
def extract_features(file_path):
    # Load audio file
    audio, sample_rate = librosa.load(file_path)
    # Extract features using Mel-Frequency Cepstral Coefficients (MFCC)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    # Flatten the features into a 1D array
    flattened_features = np.mean(mfccs.T, axis=0)
    return flattened_features

In [30]:
def audio_classification(file_path):
    extracted_features = extract_features(file_path).reshape(1, -1)
    y_predict = random_forest_model.predict_proba(extracted_features)
    
    return y_predict

In [2]:
random_forest_model = load('/kaggle/input/trained-audio-model-v-01/audio_classifier_model.joblib')

In [3]:
meta_data = pl.read_csv('/kaggle/input/birdclef-2024/train_metadata.csv', low_memory=True)
print(list(meta_data.columns))

['primary_label', 'secondary_labels', 'type', 'latitude', 'longitude', 'scientific_name', 'common_name', 'author', 'license', 'rating', 'url', 'filename']


In [4]:
bird_cols = list(meta_data['primary_label'].unique().sort())
bird_cols

['asbfly',
 'ashdro1',
 'ashpri1',
 'ashwoo2',
 'asikoe2',
 'asiope1',
 'aspfly1',
 'aspswi1',
 'barfly1',
 'barswa',
 'bcnher',
 'bkcbul1',
 'bkrfla1',
 'bkskit1',
 'bkwsti',
 'bladro1',
 'blaeag1',
 'blakit1',
 'blhori1',
 'blnmon1',
 'blrwar1',
 'bncwoo3',
 'brakit1',
 'brasta1',
 'brcful1',
 'brfowl1',
 'brnhao1',
 'brnshr',
 'brodro1',
 'brwjac1',
 'brwowl1',
 'btbeat1',
 'bwfshr1',
 'categr',
 'chbeat1',
 'cohcuc1',
 'comfla1',
 'comgre',
 'comior1',
 'comkin1',
 'commoo3',
 'commyn',
 'compea',
 'comros',
 'comsan',
 'comtai1',
 'copbar1',
 'crbsun2',
 'cregos1',
 'crfbar1',
 'crseag1',
 'dafbab1',
 'darter2',
 'eaywag1',
 'emedov2',
 'eucdov',
 'eurbla2',
 'eurcoo',
 'forwag1',
 'gargan',
 'gloibi',
 'goflea1',
 'graher1',
 'grbeat1',
 'grecou1',
 'greegr',
 'grefla1',
 'grehor1',
 'grejun2',
 'grenig1',
 'grewar3',
 'grnsan',
 'grnwar1',
 'grtdro1',
 'gryfra',
 'grynig2',
 'grywag',
 'gybpri1',
 'gyhcaf1',
 'heswoo1',
 'hoopoe',
 'houcro1',
 'houspa',
 'inbrob1',
 'indpit1',
 

In [52]:
test_soundscapes = '/kaggle/input/birdclef-2024/test_soundscapes'
#test_soundscapes = '/kaggle/input/birdclef-2024/unlabeled_soundscapes'

for path in Path(test_soundscapes).glob("*.ogg"):
    print(path)
    #print('soundscape_'+path.stem)
    print(path.stem.split("_"))
    #print(('soundscape_'+path.stem).split("_"))
    print(path.stem.split("_"))

In [53]:
test = pd.DataFrame(
     [(path.stem, *path.stem.split("_"), path) for path in Path(test_soundscapes).glob("*.ogg")],
    columns = ["filename", "name" ,"id", "path"]
)
"""test = pd.DataFrame(
     [(path.stem, *('soundscape_'+path.stem).split("_"), path) for path in Path(test_soundscapes).glob("*.ogg")],
    columns = ["filename", "name" ,"id", "path"]
)"""
print(test.shape)
print(test.head(5))

(0, 4)
Empty DataFrame
Columns: [filename, name, id, path]
Index: []


In [57]:
filenames = test.filename.values.tolist()
file_paths = test.path.values.tolist()

submission_df = pd.DataFrame(columns=['row_id']+bird_cols)
print(submission_df.shape)

(0, 183)


In [55]:
"""
for i, file_path in enumerate(file_paths):
    print(i)
    print(file_path)
    print(filenames[i])
    print("********")
    if i == 10:
        break"""

'file_paths = test.path.values.tolist()\nfor i, file_path in enumerate(file_paths):\n    print(i)\n    print(file_path)\n    print(filenames[i])\n    print("********")\n    if i == 10:\n        break'

In [58]:
for i, file in enumerate(file_paths):
    # predicted = random_forest_model.predict_proba[i]
    predicted = audio_classification(file)
    num_rows = len(predicted)
    row_ids = [f'{filenames[i]}_{(i+1)*5}' for i in range(num_rows)]
    df = pd.DataFrame(columns=['row_id']+bird_cols)
    
    df['row_id'] = row_ids
    df[bird_cols] = predicted
    
    submission_df = pd.concat([submission_df,df]).reset_index(drop=True)
    if i == 5:
        break
    # print(submission_df.shape)

In [59]:
submission_df.head(10)

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1


In [60]:
submission_df.sum(axis=1, numeric_only=True)

Series([], dtype: float64)

In [ ]:
if submission_df.shape[0] == 0:
    submission_df = pl.read_csv('/kaggle/input/birdclef-2024/sample_submission.csv', low_memory=True)
    submission_df.write_csv('submission.csv')
else:
    submission_df.to_csv('submission.csv', index=False)